## Notebook de Analisis de Afluenta

In [1]:
import pandas as pd
import numpy as np


In [2]:
!ls

afluenta.csv		    LenderAccountBug.ipynb	README.md
afluenta - Sheet1.csv	    LenderAccountMovements.xls
fastInterestAnalysis.ipynb  LenderProfitProjection.xls


In [4]:
pd.read_excel?

In [12]:
df = pd.read_csv('LenderAccountMovements.csv',index_col=None,sep="|")
df.columns

Index(['Fecha', 'Operación', 'Débitos', 'Créditos', 'Saldo'], dtype='object')

In [13]:
df.head()

,Fecha,Operación,Débitos,Créditos,Saldo
0,22/11/2016 23:24,"Comisión Afluenta: #AR-221607-G0009 (4/24) $ 6,90",0.14,NaN,21928.17
1,22/11/2016 23:24,"Retorno por cuota: #AR-221607-G0009 (4/24) $ 6,90",NaN,4.14,21928.31
2,22/11/2016 23:21,"Comisión Afluenta: #AR-301609-G0003 (2/48) $ 8,79",0.18,NaN,21924.17
3,22/11/2016 23:21,"Retorno por cuota: #AR-301609-G0003 (2/48) $ 8,79",NaN,7.11,21924.35
4,21/11/2016 10:51,Comisión Afluenta: #AR-241607-I0004 (3/24) $ 1...,0.26,NaN,21917.24


In [17]:
# get datatypes and null vals
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796 entries, 0 to 795
Data columns (total 5 columns):
Fecha        796 non-null object
Operación    796 non-null object
Débitos      400 non-null float64
Créditos     396 non-null float64
Saldo        796 non-null float64
dtypes: float64(3), object(2)
memory usage: 31.2+ KB


In [18]:
## fill null vals with 0
numeric_cols = [df.columns[-3:]]
for col in numeric_cols:
    df[col] = df[col].fillna(0)

#set datetime col
col = 'Fecha'
df[col] = pd.to_datetime(df[col])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796 entries, 0 to 795
Data columns (total 5 columns):
Fecha        796 non-null datetime64[ns]
Operación    796 non-null object
Débitos      796 non-null float64
Créditos     796 non-null float64
Saldo        796 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 31.2+ KB


In [20]:
# check accounting i.e. inputs - outputs == current_holdings
input_money = df['Créditos'].sum() 
output_money = df['Débitos'].sum()  

last_balance = df['Saldo'].iloc[0]

In [21]:
last_balance

21928.169999999998

In [23]:
input_money

22322.079999999998

In [24]:
output_money

393.90999999999997

In [22]:
input_money - output_money == last_balance

True

In [274]:
precios = table['precio']

precios = precios.str.replace('$','')
precios = precios.str.replace(' ','')
precios = precios.str.replace(',','.')
table['precio'] = precios.astype(float)
#.astype(float)

In [275]:
precio_subasta = 1020

meses_faltantes = len(table)

capital = table['precio'].sum()

print("El capital actual del portafolio es {r:.2f} y la tasa retorno esperada es {t:.4f} porque el precio de la subasta fue {s}"\
          .format(r = capital, t = capital/precio_subasta, s = precio_subasta))

meses_futuros = [2, 3, 6, 7, 9 ,12, 15, 18, 24, 36]
for mes in meses_futuros:
    print("En {m} meses recuperas el {p:.3f} % de la subasta".format(m = mes, 
                p = table.loc[0:mes, 'precio'].sum()*100/precio_subasta))
#meses_faltantes

El capital actual del portafolio es 1330.15 y la tasa retorno esperada es 1.3041 porque el precio de la subasta fue 1020
En 2 meses recuperas el 24.227 % de la subasta
En 3 meses recuperas el 36.001 % de la subasta
En 6 meses recuperas el 63.386 % de la subasta
En 7 meses recuperas el 71.189 % de la subasta
En 9 meses recuperas el 86.802 % de la subasta
En 12 meses recuperas el 100.999 % de la subasta
En 15 meses recuperas el 109.746 % de la subasta
En 18 meses recuperas el 115.451 % de la subasta
En 24 meses recuperas el 124.425 % de la subasta
En 36 meses recuperas el 130.407 % de la subasta


In [276]:
tasa = 0.4/12
tasa

0.03333333333333333

In [277]:
table['con_interes_compuesto'] = [(1 + tasa/12)**n for n in range(len(table), 0, -1)  ]
table['con_interes_compuesto'] = table['precio']*table['con_interes_compuesto']

In [278]:
table.shape

(29, 3)

In [279]:
print("El portafolio tiene {m} meses faltantes "\
          .format(m = len(table), t = capital/precio_subasta, s = precio_subasta))

for mes in meses_futuros:
    print("Con interes compuesto en {m} meses recuperas el {p:.3f} % de la subasta".format(m = mes, 
                p = table.loc[0:mes, 'con_interes_compuesto'].sum()*100/precio_subasta))

El portafolio tiene 29 meses faltantes 
Con interes compuesto en 2 meses recuperas el 26.151 % de la subasta
Con interes compuesto en 3 meses recuperas el 38.805 % de la subasta
Con interes compuesto en 6 meses recuperas el 68.087 % de la subasta
Con interes compuesto en 7 meses recuperas el 76.381 % de la subasta
Con interes compuesto en 9 meses recuperas el 92.908 % de la subasta
Con interes compuesto en 12 meses recuperas el 107.846 % de la subasta
Con interes compuesto en 15 meses recuperas el 116.965 % de la subasta
Con interes compuesto en 18 meses recuperas el 122.867 % de la subasta
Con interes compuesto en 24 meses recuperas el 132.029 % de la subasta
Con interes compuesto en 36 meses recuperas el 138.053 % de la subasta


In [283]:
flujo_prestamos = pd.read_csv('flujo-caja.csv')

In [284]:
flujo_prestamos.head()

,Período,Detalle,Capital,Interés,Int. punitorios,Comisión,Retorno neto
0,2016-07-01,13 cuotas en este período,0.00,0.00,0,0.00,0.00
1,2016-08-01,25 cuotas en este período,153.07,178.01,0,6.62,324.46
2,2016-09-01,25 cuotas en este período,158.97,172.15,0,6.62,324.50
3,2016-10-01,25 cuotas en este período,165.02,166.06,0,6.62,324.46
4,2016-11-01,25 cuotas en este período,171.40,159.69,0,6.62,324.47


In [290]:
flujo_prestamos.columns = ['Date', 'Details', 'Capital', 'Interest', 'Interest-Fine', 'Commission', 'NetReturn']

In [286]:
flujo_prestamos.dtypes

Período             object
Detalle             object
Capital            float64
Interés            float64
Int. punitorios      int64
Comisión           float64
Retorno neto       float64
dtype: object

In [291]:
flujo_prestamos['Date'] = pd.to_datetime(flujo_prestamos['Date'], yearfirst=True, infer_datetime_format=True)
#flujo_prestamos

In [293]:
tasa = 0.38
tasa = tasa/12

In [295]:
flujo_prestamos['con_interes_compuesto'] = [(1 + tasa/12)**n for n in range(len(flujo_prestamos), 0, -1)  ]
flujo_prestamos['con_interes_compuesto'] = flujo_prestamos['NetReturn']*flujo_prestamos['con_interes_compuesto']

In [296]:
precio_subasta = 5263

meses_faltantes = len(flujo_prestamos)

capital_futuro = flujo_prestamos['NetReturn'].sum()

print("El capital actual del portafolio es {r:.2f} y la tasa retorno esperada es {t:.4f} "\
          .format(r = precio_subasta, t = capital_futuro/precio_subasta, s = precio_subasta))

meses_futuros = [2, 3, 6, 7, 9 ,12, 15, 18, 24, 36]
for mes in meses_futuros:
    print("En {m} meses recuperas el {p:.3f} % de la subasta".format(m = mes, 
                p = flujo_prestamos.loc[0:mes, 'NetReturn'].sum()*100/precio_subasta))
#meses_faltantes

El capital actual del portafolio es 5263.00 y la tasa retorno esperada es 1.3265 
En 2 meses recuperas el 12.331 % de la subasta
En 3 meses recuperas el 18.496 % de la subasta
En 6 meses recuperas el 36.991 % de la subasta
En 7 meses recuperas el 43.156 % de la subasta
En 9 meses recuperas el 55.487 % de la subasta
En 12 meses recuperas el 73.982 % de la subasta
En 15 meses recuperas el 88.394 % de la subasta
En 18 meses recuperas el 102.224 % de la subasta
En 24 meses recuperas el 117.807 % de la subasta
En 36 meses recuperas el 131.585 % de la subasta


In [298]:
print("El portafolio tiene {m} meses faltantes "\
          .format(m = len(flujo_prestamos), t = capital_futuro/precio_subasta, s = precio_subasta))

for mes in meses_futuros:
    print("Con interes compuesto en {m} meses recuperas el {p:.3f} % del capital inicial".format(m = mes, 
                p = flujo_prestamos.loc[0:mes, 'con_interes_compuesto'].sum()*100/precio_subasta))

El portafolio tiene 49 meses faltantes 
Con interes compuesto en 2 meses recuperas el 13.975 % del capital inicial
Con interes compuesto en 3 meses recuperas el 20.934 % del capital inicial
Con interes compuesto en 6 meses recuperas el 41.704 % del capital inicial
Con interes compuesto en 7 meses recuperas el 48.591 % del capital inicial
Con interes compuesto en 9 meses recuperas el 62.310 % del capital inicial
Con interes compuesto en 12 meses recuperas el 82.754 % del capital inicial
Con interes compuesto en 15 meses recuperas el 98.560 % del capital inicial
Con interes compuesto en 18 meses recuperas el 113.607 % del capital inicial
Con interes compuesto en 24 meses recuperas el 130.367 % del capital inicial
Con interes compuesto en 36 meses recuperas el 144.875 % del capital inicial
